In [14]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import nltk, re, string
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
wordnet_lemmatizer = WordNetLemmatizer()

def coindesk_market_parser():
    headlines=[]

    page_url= "https://www.coindesk.com/category/markets-news/markets-markets-news/markets-bitcoin/"

    while page_url!=None:
        page = requests.get(page_url) 

        if page.status_code!=200:
             page_url=None
        else:
            all_data = []
            for num in range(1,20):
                page_url = "https://www.coindesk.com/category/markets-news/markets-markets-news/markets-bitcoin/page/"+str(num)+"/"
                page = requests.get(page_url)
                soup = BeautifulSoup(page.content, "html.parser")
                titles = soup.find_all("a", class_ = "fade")
                titles_dictionary = []
                for i in titles:
                    title = i.get_text().lower()
                    titles_dictionary.append(lemmatize(title))
                dates = soup.find_all("time")
                dates_list = []
                for i in dates:
                    date = i.get_text().lower()
                    date = date.replace(' at', '')
                    date = datetime.strptime(date, '%b %d, %Y %H:%M')
                    date = datetime.strftime(date,'%b-%d-%Y')
                    dates_list.append(date)
                data = zip(titles_dictionary,dates_list)  
                all_data.extend(data)

        page_url = None
    return all_data

# data_frame = pd.DataFrame.from_records(all_data, columns = ["Tokenized Title","Date"])
# data_frame.to_csv("coindesk_markets.csv", encoding='utf-8')